In [1]:
import torch
import torch.nn as nn

# 시퀀스 설정
seq_len = 5
batch_size = 1
input_size = 3
hidden_size = 4

# 입력 텐서: (seq_len, batch_size, input_size)
x_seq = torch.randn(seq_len, batch_size, input_size)

In [2]:
x_seq

tensor([[[ 0.0977,  0.4379,  0.3927]],

        [[-0.3681,  0.3229,  0.3061]],

        [[ 0.0067,  1.0418, -1.0231]],

        [[-0.3605, -0.5158,  0.4458]],

        [[ 0.6342,  1.6602, -1.6594]]])

In [3]:
x_seq.shape

torch.Size([5, 1, 3])

In [4]:
rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size)

# 초기 은닉 상태 (0으로 초기화)
h0 = torch.zeros(1, batch_size, hidden_size)

# 순전파
output, hn = rnn(x_seq, h0)

print(output.shape)  # torch.Size([seq_len, batch_size, hidden_size])
print(hn.shape)      # torch.Size([1, batch_size, hidden_size])

torch.Size([5, 1, 4])
torch.Size([1, 1, 4])


In [5]:
rnn

RNN(3, 4)

In [7]:
rnn.all_weights

[[Parameter containing:
  tensor([[-0.1298,  0.1970, -0.4207],
          [-0.3869,  0.0454, -0.0530],
          [-0.3612, -0.3522, -0.3319],
          [ 0.3288,  0.3879, -0.2173]], requires_grad=True),
  Parameter containing:
  tensor([[ 0.3564,  0.0406, -0.2110, -0.2669],
          [ 0.1593, -0.3393, -0.0758,  0.2842],
          [ 0.4191, -0.1076, -0.2189,  0.0589],
          [ 0.2722,  0.2923, -0.3402,  0.1434]], requires_grad=True),
  Parameter containing:
  tensor([-0.0682, -0.2378,  0.3546, -0.4159], requires_grad=True),
  Parameter containing:
  tensor([-0.1233,  0.1816, -0.2381,  0.4641], requires_grad=True)]]

## PyTorch RNN 한 시점의 연산식

PyTorch의 `nn.RNN`은 한 시점에서 다음과 같은 연산을 수행합니다.

$$
h_t = \tanh(x_t W_{ih}^T + b_{ih} + h_{t-1} W_{hh}^T + b_{hh})
$$

---

### 파라미터의 저장 형태와 연산 시 전치 형태

| 파라미터 이름 | 저장 시 shape | 연산 시 전치 후 shape | 설명 |
|----------------|----------------|----------------------|------|
| $W_{ih}$ (`weight_ih_l0`) | (hidden_size, input_size) = (4, 3) | (input_size, hidden_size) = (3, 4) | 입력 $x_t$ 에 곱해질 때 전치됨 |
| $W_{hh}$ (`weight_hh_l0`) | (hidden_size, hidden_size) = (4, 4) | (hidden_size, hidden_size) = (4, 4) | 자기 자신과 곱하므로 전치되어도 동일 |
| $b_{ih}$ (`bias_ih_l0`) | (hidden_size,) = (4,) | (4,) | 그대로 더해짐 |
| $b_{hh}$ (`bias_hh_l0`) | (hidden_size,) = (4,) | (4,) | 그대로 더해짐 |

---

### 입력과 출력의 크기

| 항목 | 의미 | shape |
|------|------|--------|
| $x_t$ | 현재 시점 입력 | (batch_size, input_size) = (1, 3) |
| $h_{t-1}$ | 이전 은닉 상태 | (batch_size, hidden_size) = (1, 4) |
| $h_t$ | 현재 은닉 상태 (출력) | (batch_size, hidden_size) = (1, 4) |

---

### 실제 연산 과정 형태 표시

$$
\begin{aligned}
h_t &= \tanh( x_t W_{ih}^{T} + b_{ih} + h_{t-1} W_{hh}^{T} + b_{hh}) \\
&= \tanh( (1\times3)\,(3\times4) + (4,) + (1\times4)\,(4\times4) + (4,) ) \\
&\Rightarrow h_t \in \mathbb{R}^{1\times4}
\end{aligned}
$$

---




In [ ]:
h0

tensor([[[0., 0., 0., 0.]]])

In [ ]:
h0.shape

torch.Size([1, 1, 4])

## PyTorch RNN의 초기 은닉 상태 h₀의 shape

RNN은 첫 시점에서 과거 은닉 상태가 없기 때문에,  
사용자가 초기 은닉 상태 $h_0$를 직접 만들어 넣어줍니다.

```python
h0 = torch.zeros(1, batch_size, hidden_size)


| 차원                    | 값                           | 의미                                           |
| --------------------- | --------------------------- | -------------------------------------------- |
| 첫 번째 차원 = 1           | num_layers × num_directions | 단일층(single layer) × 단방향(unidirectional)이므로 1 |
| 두 번째 차원 = batch_size  | 현재 배치의 개수                   | 예: 1개 시퀀스(문장)                                |
| 세 번째 차원 = hidden_size | 은닉 상태의 차원 크기                | 예: 4                                         |


## RNN의 출력 텐서 output

```python
print(output.shape)  # torch.Size([5, 1, 4])


| 항목       | shape                              | 실제 값                  | 의미                        |
| -------- | ---------------------------------- | --------------------- | ------------------------- |
| `output` | (seq_len, batch_size, hidden_size) | torch.Size([5, 1, 4]) | 모든 시점의 은닉 상태 $h_t$를 쌓은 텐서 |


RNN은 입력 시퀀스의 각 시점 $t$마다 은닉 상태 $h_t$를 계산합니다.  
PyTorch는 이 **모든 시점의 결과**를 하나의 3차원 텐서로 묶어 반환합니다.

$$
output = [h_1, h_2, h_3, h_4, h_5]
$$


seq_len = 5, batch_size = 1, hidden_size = 4 라면  

$$
output.shape = (5,\, 1,\, 4)
$$

즉,  
**5개의 시점 × 1개의 배치 × 4차원 은닉 상태**를 의미합니다.


In [ ]:
lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size)

# 초기 상태 설정 (은닉 상태 + 셀 상태)
h0 = torch.zeros(1, batch_size, hidden_size)
c0 = torch.zeros(1, batch_size, hidden_size)

# 순전파
output, (hn, cn) = lstm(x_seq, (h0, c0))

print(output.shape)  # (seq_len, batch_size, hidden_size)
print(hn.shape)      # (1, batch_size, hidden_size)
print(cn.shape)      # (1, batch_size, hidden_size)

torch.Size([5, 1, 4])
torch.Size([1, 1, 4])
torch.Size([1, 1, 4])


## LSTM의 출력 텐서 구조

| 텐서 | shape | 의미 |
|------|--------|------|
| `output` | `(5, 1, 4)` | **5개 시점의 모든 은닉 상태** → 각 $t$마다 $h_t$ 포함 |
| `hn` | `(1, 1, 4)` | **마지막 시점($t=5$)**의 은닉 상태 $h_T$<br>첫 번째 차원 `1`은 **레이어 수 × 방향 수** (단일층, 단방향이므로 1) |
| `cn` | `(1, 1, 4)` | **마지막 시점($t=5$)**의 셀 상태 $c_T$<br>첫 번째 차원 `1`은 **레이어 수 × 방향 수** (단일층, 단방향이므로 1) |

---

### 개념

$$
\text{output} =
\begin{bmatrix}
\boldsymbol{h}_1 \\
\boldsymbol{h}_2 \\
\boldsymbol{h}_3 \\
\boldsymbol{h}_4 \\
\boldsymbol{h}_5
\end{bmatrix},
\quad
\text{hn} = \boldsymbol{h}_5,
\quad
\text{cn} = \boldsymbol{c}_5
$$

---

즉,  
- `output` → **모든 시점의 은닉 상태**  
- `hn` → **마지막 시점의 은닉 상태**  
- `cn` → **마지막 시점의 셀 상태**  
- 첫 번째 차원 `1`은 `num_layers × num_directions = 1 × 1`  
  → 단일층, 단방향 LSTM을 의미합니다.
